# User routes on the site
## Description
**Clickstream** is a sequence of user actions on a website. It allows you to understand how users interact with the site. In this task, you need to find the most frequent custom routes.

## Input data
Input data is а table with clickstream data in file `hdfs:/data/clickstream.csv`.

### Table structure
* `user_id (int)` - Unique user identifier.
* `session_id (int)` - Unique identifier for the user session. The user's session lasts until the identifier changes.
* `event_type (string)` - Event type from the list:
    * **page** - visit to the page
    * **event** - any action on the page
    * <b>&lt;custom&gt;</b> - string with any other type
* `event_type (string)` - Page on the site.
* `timestamp (int)` - Unix-timestamp of action.

### Browser errors
Errors can sometimes occur in the user's browser - after such an error appears, we can no longer trust the data of this session and all the following lines after the error or at the same time with it are considered corrupted and **should not be counted** in statistics.

When an error occurs on the page, a random string containing the word **error** will be written to the `event_type` field.

### Sample of user session
<pre>
+-------+----------+------------+----------+----------+
|user_id|session_id|  event_type|event_page| timestamp|
+-------+----------+------------+----------+----------+
|    562|       507|        page|      main|1620494781|
|    562|       507|       event|      main|1620494788|
|    562|       507|       event|      main|1620494798|
|    562|       507|        page|    family|1620494820|
|    562|       507|       event|    family|1620494828|
|    562|       507|        page|      main|1620494848|
|    562|       507|wNaxLlerrorU|      main|1620494865|
|    562|       507|       event|      main|1620494873|
|    562|       507|        page|      news|1620494875|
|    562|       507|        page|   tariffs|1620494876|
|    562|       507|       event|   tariffs|1620494884|
|    562|       514|        page|      main|1620728918|
|    562|       514|       event|      main|1620729174|
|    562|       514|        page|   archive|1620729674|
|    562|       514|        page|     bonus|1620729797|
|    562|       514|        page|   tariffs|1620731090|
|    562|       514|       event|   tariffs|1620731187|
+-------+----------+------------+----------+----------+
</pre>

#### Correct user routes for a given user:
* **Session 507**: main-family-main
* **Session 514**: main-archive-bonus-tariffs

Route elements are ordered by the time they appear in the clickstream, from earliest to latest.

The route must be accounted for completely before the end of the session or an error in the session.

## Task
You need to use the Spark SQL, Spark RDD and Spark DF interfaces to create a solution file, the lines of which contain **the 30 most frequent user routes** on the site.

Each line of the file should contain the `route` and `count` values **separated by tabs**, where:
* `route` - route on the site, consisting of pages separated by "-".
* `count` - the number of user sessions in which this route was.

The lines must be **ordered in descending order** of the `count` field.

## Criteria
You can get maximum of 3.5 points (final grade) for this assignment, depedning on the number of interface you manage to leverage. The criteria are as follows:

* 0.5 points – Spark SQL solution with 1 query
* 0.5 points – Spark SQL solution with <=2 queries
* 0.5 points – Spark RDD solution
* 0.5 points – Spark DF solution
* 0.5 points – your solution algorithm is relatively optimized, i.e.: no O^2 or O^3 complexities; appropriate object usage; no data leaks etc. This is evaluated by staff.
* 1 point – 1 on 1 screening session. During this session staff member can ask you questions regarding your solution logic, framework usage, questionable parts of your code etc. If your code is clean enough, the staff member can just ask you to solve a theoretical problem connected to Spark.


In [1]:
import findspark
findspark.init()

import pyspark
sc = pyspark.SparkContext(appName='jupyter')

from pyspark.sql import SparkSession, Row
se = SparkSession(sc)


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
2024-01-21 19:28:45,007 WARN yarn.Client: Neither spark.yarn.jars nor spark.yarn.archive is set, falling back to uploading libraries under SPARK_HOME.


In [2]:
df = se.read.csv("hdfs:/data/clickstream.csv", sep ='\t', header=True)
df.registerTempTable("df")


In [3]:
import pyspark.sql.functions as F
from pyspark.sql.window import Window
import numpy as np

In [29]:
solution = df.select(F.col("*"), F.when(F.col("event_page") == F.lag("event_page").over(Window.orderBy("timestamp",   "user_id", "session_id")), None)
                 .otherwise(F.col("event_page"))
                 .alias("case")).filter(F.col("case").isNull() == False).groupBy("user_id", "session_id").agg(F.collect_list("event_page").alias("route")).withColumn("route", F.concat_ws("-", "route")).groupby("route").count().orderBy("count", ascending=False).limit(30)

In [30]:
solution.write.option("header",True).csv("resultsql.csv")

2024-01-21 20:11:06,159 WARN window.WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.


AnalysisException: path hdfs://localhost:9000/user/jovyan/resultsql.csv already exists.

In [34]:
solution.show(30)

2024-01-21 20:14:00,425 WARN window.WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
2024-01-21 20:14:02,707 WARN window.WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.


+--------------------+-----+
|               route|count|
+--------------------+-----+
|                main| 2653|
|           main-main| 1428|
|      main-main-main|  800|
| main-main-main-main|  441|
|main-main-main-ma...|  254|
|        main-archive|  174|
|         main-rabota|  134|
|       main-internet|  125|
|          main-bonus|  114|
|main-main-main-ma...|  112|
|           main-news|  106|
|main-archive-archive|   90|
|main-internet-int...|   90|
|        main-tariffs|   89|
|  main-rabota-rabota|   86|
|         main-online|   84|
|    main-main-rabota|   82|
|   main-main-archive|   81|
|          main-vklad|   73|
|  main-main-internet|   72|
|     main-main-bonus|   71|
|    main-bonus-bonus|   68|
|      main-news-news|   66|
|main-main-main-ma...|   64|
|main-main-main-ar...|   61|
|      main-main-news|   59|
|main-tariffs-tariffs|   58|
|main-archive-arch...|   54|
|    main-main-online|   54|
|main-main-archive...|   53|
+--------------------+-----+

